In [1]:
from analysis.load_spectra import load_acetonitrile_spectra
from lmfit.models import VoigtModel
from ramanalysis.calibrate import ACETONITRILE_PEAKS_CM1

# Load acetonitrile spectra and their corresponding spectrometers
acetonitrile_spectra, acetonitrile_dataframe = load_acetonitrile_spectra()

# Set parameters for peak finding
num_peaks = 4
window_size = 25
wavenumber_range_cm1 = (750, 3100)
model_class = VoigtModel

In [2]:
# | code-fold: true
# | code-summary: "Source code for this figure:"
# | label: fig-acetonitrile
# | fig-cap: "Acetonitrile spectra from each instrument. The four most prominent peaks for each spectrum are detected and fit using a Voigt profile. Hovering the cursor over each peak lets you see the measured FWHM, height, and offset from the reference peak value."

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from analysis.plotting import darken, get_custom_colorpalette, get_default_plotly_layout
from plotly.subplots import make_subplots
from ramanalysis.peak_fitting import find_n_most_prominent_peaks

# Store peak fitting results
peak_fitting_records = []

# Create figure
color_palette = get_custom_colorpalette()
num_rows = acetonitrile_dataframe.groupby(["λ_nm"]).count()["instrument"].max().item()
num_cols = acetonitrile_dataframe["λ_nm"].nunique()
fig = make_subplots(
    rows=num_rows,
    cols=num_cols,
    shared_xaxes=True,
    shared_yaxes=True,
    horizontal_spacing=0.02,
    vertical_spacing=0.05,
    column_titles=["785 nm", "532 nm"],
)

# Map each (instrument, wavelength) combo to a subplot
subplot_indices = {  # (instrument, λ_nm) -> (col, row)
    ("horiba", 785): (1, 1),
    ("renishaw", 785): (1, 2),
    ("wasatch", 785): (1, 3),
    ("openraman", 532): (2, 2),
    ("wasatch", 532): (2, 3),
}

for i, dataframe_row in acetonitrile_dataframe.iterrows():
    # Get the raw spectrum and the instrument and wavelength with which it was recorded
    wavelength_nm = dataframe_row["λ_nm"]
    instrument = dataframe_row["instrument"]
    raw_spectrum = acetonitrile_spectra[i]
    subplot_col, subplot_row = subplot_indices[(instrument, wavelength_nm)]

    # Apply median filtering on OpenRAMAN data to filter out hot pixels from the detector
    if instrument == "openraman":
        raw_spectrum = raw_spectrum.smooth(kernel_size=3)
    # Crop and normalize spectra for peak detection
    preprocessed_spectrum = raw_spectrum.between(*wavenumber_range_cm1).normalize()

    # Plot each acetonitrile spectrum
    fig.add_trace(
        go.Scatter(
            x=preprocessed_spectrum.wavenumbers_cm1,
            y=preprocessed_spectrum.intensities,
            name=instrument,
            hoverinfo="skip",
            marker={"color": color_palette[(instrument, wavelength_nm)]},
        ),
        row=subplot_row,
        col=subplot_col,
    )

    # Find and get the indices of the most prominent peaks in the spectrum
    peak_indices = find_n_most_prominent_peaks(
        preprocessed_spectrum.intensities, num_peaks=num_peaks
    )
    for i_peak in peak_indices:
        # Create window around each peak
        peak_cm1, peak_height = preprocessed_spectrum.interpolate(i_peak)
        window_range = (i_peak - window_size / 2, i_peak + window_size / 2)
        window = np.linspace(*window_range, num=500)
        # Interpolate the window along the spectrum
        x_data_cm1, y_data = preprocessed_spectrum.interpolate(window)

        # Fit the model to each peak
        model = model_class()
        initial_fit_params = model.make_params(amplitude=1, center=peak_cm1, sigma=1, gamma=0.5)
        fit_result = model.fit(y_data, initial_fit_params, x=x_data_cm1)

        # Find the offset from acetonitrile reference peaks
        peak_center = fit_result.params["center"].value
        reference_peak_index = np.abs(ACETONITRILE_PEAKS_CM1 - peak_center).argmin()
        reference_peak = ACETONITRILE_PEAKS_CM1[reference_peak_index]
        offset = peak_center - reference_peak

        # Record the peak fitting results
        peak_fitting_results = {**dataframe_row.to_dict(), **fit_result.params.valuesdict()}
        peak_fitting_results["reference"] = reference_peak
        peak_fitting_results["offset"] = offset
        peak_fitting_records.append(peak_fitting_results)

        # Plot and annotate each peak
        annotation_text = f"{peak_fitting_results['fwhm']:.1f} cm<sup>-1</sup>"
        hovertext_fields = ["fwhm", "height", "offset"]
        hovertext = [
            f"{field}: {peak_fitting_results[field]:.3g}<br>" for field in hovertext_fields
        ]
        fig.add_trace(
            go.Scatter(
                x=[peak_center],
                y=[peak_fitting_results["height"]],
                showlegend=False,
                hovertext="".join(hovertext),
                text=annotation_text,
                mode="markers+text",
                textposition="top center",
                name=instrument,
                marker={"color": darken(color_palette[(instrument, wavelength_nm)])},
            ),
            row=subplot_row,
            col=subplot_col,
        )

        # Plot fitted Voigt profile for each peak
        x_data = np.linspace(peak_center - window_size, peak_center + window_size, 500)
        model_params = [peak_fitting_results[p] for p in ["amplitude", "center", "sigma", "gamma"]]
        y_data = model.func(x_data, *model_params)
        fig.add_trace(
            go.Scatter(
                x=x_data,
                y=y_data,
                showlegend=False,
                hoverinfo="skip",
                marker={"color": darken(color_palette[(instrument, wavelength_nm)])},
            ),
            row=subplot_row,
            col=subplot_col,
        )

# Create pandas DataFrame of peak fitting results
acetonitrile_peaks_dataframe = pd.DataFrame.from_records(peak_fitting_records).drop("gamma", axis=1)

# Configure plotly layout
layout = get_default_plotly_layout()
fig.update_layout(
    layout,
    height=450,
)
fig.update_xaxes(range=[wavenumber_range_cm1[0] - 100, wavenumber_range_cm1[1] + 100])
fig.update_yaxes(range=[-0.05, 1.3])
fig.update_xaxes(title_text="Wavenumber (cm<sup>-1</sup>)", row=num_rows, col=1)
fig.update_xaxes(title_text="Wavenumber (cm<sup>-1</sup>)", row=num_rows, col=2)
fig.update_yaxes(title_text="Intensity (normalized)", row=2, col=1)
fig.show()

### Characterizing SNR with *C. reinhardtii* spectra

We're shifting our focus from a standard sample (acetonitrile) to biological samples, specifically unicellular algae, to better understand how the different spectrometers perform for our experiments of interest. To do this, we collected Raman spectra of *Chlamydomonas reinhardtii* ([CC-124](https://www.chlamycollection.org/product/cc-124-wild-type-mt-137c/))—a model organism we frequently use in the lab for studying cell motility—from each instrument. Since biological samples introduce more complexity than pure compounds, we need to assess the signal-to-noise ratio (SNR) of these spectra to determine which instruments offer the highest sensitivity and are best suited for detecting subtle spectral features. In addition to SNR measurements, we’ll make (relatively crude) peak-finding calculations to get a rough sense of the number of spectral features we can resolve from each spectrum. To find peaks, we'll use the [`find_peaks`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks.html) function from [`Scipy`](https://scipy.org/) (version 1.14.1) and define the relative prominence and the separation of the peaks we're interested in resolving.

We’ll also need to perform baseline subtraction to derive meaningful SNR measurements, as raw Raman spectra often contain fluorescence and other background signals that obscure meaningful peaks. In a separate analysis, we evaluated different baseline estimation algorithms and found that adaptive smoothness penalized least squares (asPLS) performed the best in terms of removing background interference while preserving key spectral features [@zhang2020baseline]. We use the implementation of asPLS in [`pybaselines`](https://pybaselines.readthedocs.io/en/latest/api/pybaselines/whittaker/index.html#pybaselines.whittaker.aspls) with the smoothing parameter changed to `lam = 1e4`, as we found empirically that this value provided better baseline estimation for our dataset.

In [4]:
from analysis.load_spectra import load_cc124_tap_spectra
from pybaselines.whittaker import aspls
from ramanalysis import RamanSpectrum
from scipy.signal import find_peaks

cc124_tap_spectra, cc124_tap_dataframe = load_cc124_tap_spectra()


# Define a function for calculating peak SNR
def compute_peak_snr(
    spectrum: RamanSpectrum,
    noisy_region_cm1: tuple[float, float],
) -> float:
    """Peak SNR measurement."""
    noise = spectrum.normalize().between(*noisy_region_cm1).intensities.std()
    peak_snr = 1 / noise
    return peak_snr


# Parameters for baseline subtraction
lam = 1e4
wavenumber_range_cm1 = (520, 3200)

# Parameters for SNR calculation
noisy_region_cm1 = (1700, 2500)

# Parameters for peak finding
relative_prominences = {
    ("horiba", 785): 0.5,
    ("renishaw", 785): 0.1,
    ("wasatch", 785): 0.1,
    ("openraman", 532): 0.5,
    ("wasatch", 532): 0.1,
}
peak_separation = 10

@fig-cc124 shows the peak SNR measurements of each instrument.

In [5]:
# | code-fold: true
# | code-summary: "Source code for this figure:"
# | label: fig-cc124
# | fig-cap: "Peak SNR measurements and detected peaks in *C. reinhardtii* spectra from each instrument. Each subplot shows the baseline-subtracted spectrum on which the peak SNR measurements are made. Behind the baseline-subtracted spectra, the raw *C. reinhardtii* spectra and estimated baselines are shown with semi-transparency. Note that the estimated baselines are slightly recessed from the raw spectra for visualization. Clicking on the instrument name in the legend toggles the display of the baseline-subtracted spectra to view the raw spectrum and estimated baseline more easily."

# Store peak SNR measurement results
snr_measurements = []

# Create figure
num_rows = cc124_tap_dataframe.groupby(["λ_nm"]).count()["instrument"].max().item()
num_cols = cc124_tap_dataframe["λ_nm"].nunique()
fig = make_subplots(
    rows=num_rows,
    cols=num_cols,
    shared_xaxes=True,
    shared_yaxes=True,
    horizontal_spacing=0.02,
    vertical_spacing=0.05,
    column_titles=["785 nm", "532 nm"],
)

# Map each (instrument, wavelength) combo to a subplot
subplot_indices = {  # (instrument, λ_nm) -> (col, row)
    ("horiba", 785): (1, 1),
    ("renishaw", 785): (1, 2),
    ("wasatch", 785): (1, 3),
    ("openraman", 532): (2, 2),
    ("wasatch", 532): (2, 3),
}

for i, dataframe_row in cc124_tap_dataframe.iterrows():
    # Get the raw spectrum and the instrument and wavelength with which it was recorded
    wavelength_nm = dataframe_row["λ_nm"]
    instrument = dataframe_row["instrument"]
    raw_spectrum = cc124_tap_spectra[i]
    subplot_col, subplot_row = subplot_indices[(instrument, wavelength_nm)]

    # Crop and normalize spectra for baseline estimation
    normalized_spectrum = raw_spectrum.between(*wavenumber_range_cm1).normalize()
    # Apply median filtering on OpenRAMAN data to filter out hot pixels from the detector
    if instrument == "openraman":
        normalized_spectrum = normalized_spectrum.smooth(kernel_size=3)

    # Estimate and subtract baseline
    baseline_estimate, _params = aspls(normalized_spectrum.intensities, lam=lam)
    baseline_subtracted_intensities = normalized_spectrum.intensities - baseline_estimate
    baseline_subtracted_spectrum = RamanSpectrum(
        wavenumbers_cm1=normalized_spectrum.wavenumbers_cm1,
        intensities=baseline_subtracted_intensities,
    ).normalize()

    # Crop out saturated region from Wasatch 532 nm spectrum
    if (instrument == "wasatch") and (wavelength_nm == 532):
        baseline_subtracted_spectrum = baseline_subtracted_spectrum.between(0, 1800).normalize()

    # Calculate peak SNR
    snr = compute_peak_snr(baseline_subtracted_spectrum, noisy_region_cm1)
    snr_measurements.append(snr)

    # Find prominent peaks
    peaks, _peak_properties = find_peaks(
        baseline_subtracted_spectrum.intensities,
        relative_prominences[(instrument, wavelength_nm)],
        distance=peak_separation,
    )
    peaks_cm1, peak_heights = baseline_subtracted_spectrum.interpolate(peaks)

    # Plot each raw (normalized) spectrum
    fig.add_trace(
        go.Scatter(
            x=normalized_spectrum.wavenumbers_cm1,
            y=normalized_spectrum.intensities,
            hoverinfo="skip",
            showlegend=False,
            marker={"color": color_palette[(instrument, wavelength_nm)]},
            opacity=0.5,
        ),
        row=subplot_row,
        col=subplot_col,
    )

    # Plot each baseline estimate
    fig.add_trace(
        go.Scatter(
            x=normalized_spectrum.wavenumbers_cm1,
            y=baseline_estimate - 0.05,
            hoverinfo="skip",
            showlegend=False,
            marker={"color": darken(color_palette[(instrument, wavelength_nm)])},
            opacity=0.5,
        ),
        row=subplot_row,
        col=subplot_col,
    )

    # Plot baseline subtracted spectra
    name = f"{instrument.capitalize()} | {wavelength_nm} nm"
    fig.add_trace(
        go.Scatter(
            x=baseline_subtracted_spectrum.wavenumbers_cm1,
            y=baseline_subtracted_spectrum.intensities,
            name=name,
            legendgroup=name,
            hoverinfo="skip",
            marker={"color": color_palette[(instrument, wavelength_nm)]},
        ),
        row=subplot_row,
        col=subplot_col,
    )

    # Plot peaks
    fig.add_trace(
        go.Scatter(
            x=peaks_cm1,
            y=peak_heights,
            name=name,
            legendgroup=name,
            mode="markers",
            marker={"color": color_palette[(instrument, wavelength_nm)]},
            showlegend=False,
        ),
        row=subplot_row,
        col=subplot_col,
    )

    # Plot SNR and num peaks annotation
    text = f"Peak SNR: {snr:.1f}<br>Num peaks: {peaks.size:.0f}"
    fig.add_trace(
        go.Scatter(
            x=[2000],
            y=[0.6],
            showlegend=False,
            text=text,
            mode="text",
            textposition="middle right",
            name=instrument,
        ),
        row=subplot_row,
        col=subplot_col,
    )

# Create pandas DataFrame of peak fitting results
cc124_tap_dataframe["snr"] = snr_measurements

# Configure plotly layout
fig.update_layout(
    layout,
    height=450,
)
fig.update_xaxes(range=[wavenumber_range_cm1[0] - 100, wavenumber_range_cm1[1] + 100])
fig.update_yaxes(range=[-0.05, 1.3])
fig.update_xaxes(title_text="Wavenumber (cm<sup>-1</sup>)", row=num_rows, col=1)
fig.update_xaxes(title_text="Wavenumber (cm<sup>-1</sup>)", row=num_rows, col=2)
fig.update_yaxes(title_text="Intensity (normalized)", row=2, col=1)
fig.show()

These measurements indicate that the Renishaw and 785 nm Wasatch systems provide the highest peak SNR values. The OpenRAMAN exhibits the lowest SNR, which aligns with expectations given its relatively low-cost components. Perhaps not surprisingly, the two instruments with the highest SNR also have the highest number of discernible spectral features. While it also has a relatively high SNR, the number of features in the 532 nm Wasatch spectrum is much lower. This suggests that 532 nm may be less optimal for analyzing these particular types of samples. It should be noted that the number of peaks detected in each spectrum is only an approximation.

Now, we’ll move on to applications requiring large spectral datasets, such as batch processing and machine learning (ML) techniques. These approaches will help us evaluate the reproducibility of each instrument and assess their sensitivity to subtle variations in, e.g., differentiating strains and growth conditions of our cell samples.

### Batch preprocessing of *Chlamydomonas* spectra {#sec-batchpreprocessing}

The *C. reinhardtii* spectra characterized above come from a much larger dataset of *Chlamydomonas* spectra. Three strains of *Chlamydomonas* (representing two species) were cultured in two different media conditions, as described in @sec-sampleprep. With each instrument, we acquired Raman spectra from plates (or, in the case of the Renishaw, glass slides) of cells representing each of these six treatments. Since these spectra capture biochemical variations between species and environmental influences, we aim to apply machine-learning techniques to distinguish between groups. However, before diving into the modeling, we need to perform batch preprocessing to correct for background fluorescence, read noise from the detectors, and other experimental inconsistencies. This ensures that our spectral differences reflect biological variability (as much as possible) rather than artifacts from acquisition conditions, improving the reliability of our classification models. Again, we're aiming for uniform treatment of the spectra so we can pull out relative differences rather than preventing any slight shifts from the original peak positions. The preprocessing pipeline is comprised of the following steps:

1. **Denoising**: Smooth spectra with [Savitzky-Golay filter](https://en.wikipedia.org/wiki/Savitzky%E2%80%93Golay_filter).
2. **Cropping**: Crop spectra to the fingerprint region (300–1,800 cm^-1^).
3. **Baseline subtraction**: Fit and remove the fluorescence baseline using the asPLS algorithm.
4. **Normalization**: Apply min-max normalization to scale the spectra for consistency across samples.

For the Renishaw, we need to perform an additional background subtraction step due to the use of a glass slide as the sample substrate. The Raman signal from the glass introduces a distinct background that overlaps with our biological spectra [@fikiet2020clarifying], requiring a separate correction to prevent it from confounding our analysis.

# Quick analysis

In [4]:
from analysis.plotting import darken, get_custom_colorpalette, get_default_plotly_layout
from analysis.load_spectra import load_chlamy_spectra
from scipy.signal import savgol_filter
import plotly.express as px

# Configure plotly layout
layout = get_default_plotly_layout()

# Parameters for baseline subtraction
lam = 1e4
wavenumber_range_cm1 = (520, 3200)

# Parameters for SNR calculation
noisy_region_cm1 = (1700, 2500)

# Parameters for peak finding
relative_prominences = {
    ("horiba", 785): 0.5,
    ("renishaw", 785): 0.1,
    ("wasatch", 785): 0.1,
    ("openraman", 532): 0.5,
    ("wasatch", 532): 0.1,
}
peak_separation = 10


# Load all the spectra of Chlamydomonas and the corresponding metadata
data_dir = '/Volumes/Microscopy/Wasatch-Raman_785/20250306/test2'
chlamy_spectra, chlamy_dataframe = load_chlamy_spectra(data_dir)

# Preprocessing parameters
savgol_window_length = 9
savgol_polynomial_order = 3
fingerprint_region_cm1 = (300, 1800)

# Show a sample of spectra metadata
print(f"Total number of Chlamydomonas spectra: {len(chlamy_dataframe)}")

# one color for each well
color_palette = px.colors.sample_colorscale("Viridis", len(chlamy_dataframe.well_ID.unique()))
color_map = {well: color_palette[i] for i, well in enumerate(chlamy_dataframe.well_ID.unique())}
chlamy_dataframe.sample(10, random_state=57)

Total number of Chlamydomonas spectra: 288


,instrument,λ_nm,well_ID,site
51,wasatch,785,B4,3
222,wasatch,785,G2,2
8,wasatch,785,A3,0
119,wasatch,785,D3,3
122,wasatch,785,D4,2
237,wasatch,785,G6,1
170,wasatch,785,E7,2
77,wasatch,785,C2,1
271,wasatch,785,H5,3
20,wasatch,785,A6,0


In [ ]:
from ramanalysis import RamanSpectrum
from pybaselines.whittaker import aspls
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd

# Load spectrum from Renishaw glass slide control
txt_filepath = "data/Renishaw_Qontor/glass_slide_background.txt"
glass_slide_control_spectrum = RamanSpectrum.from_renishaw_txtfile(txt_filepath)

# Store preprocessed spectra
preprocessed_chlamy_spectra = []

for i, dataframe_row in chlamy_dataframe.iterrows():
    # Get the raw spectrum and its associated metadata
    raw_spectrum = chlamy_spectra[i]
    wavelength_nm = dataframe_row["λ_nm"]
    instrument = dataframe_row["instrument"]
    well_ID = dataframe_row["well_ID"]
    site = dataframe_row["site"]

    # Preprocessing steps
    # -------------------
    # 1) Denoising
    smoothed_intensities = savgol_filter(
        raw_spectrum.intensities,
        window_length=savgol_window_length,
        polyorder=savgol_polynomial_order,
    )
    # 1.5) Glass slide background subtraction for the Renishaw
    if instrument == "renishaw":
        smoothed_intensities -= glass_slide_control_spectrum.intensities
    # 2) Crop spectral range to the fingerprint region
    cropped_spectrum = RamanSpectrum(
        wavenumbers_cm1=raw_spectrum.wavenumbers_cm1,
        intensities=smoothed_intensities,
    ).between(*fingerprint_region_cm1)
    # 2.5) Additional cropping for 532 nm Wasatch
    if (instrument == "wasatch") and (wavelength_nm == 532):
        cropped_spectrum = cropped_spectrum.between(0, 1700)
    # 3) Baseline subtraction
    baseline_estimate, _params = aspls(cropped_spectrum.intensities, lam=lam)
    baseline_subtracted_intensities = cropped_spectrum.intensities - baseline_estimate
    # 4) Compose into RamanSpectrum and normalize
    preprocessed_spectrum = RamanSpectrum(
        wavenumbers_cm1=cropped_spectrum.wavenumbers_cm1,
        intensities=baseline_subtracted_intensities,
    ).normalize()
    preprocessed_chlamy_spectra.append(preprocessed_spectrum)

# Create figure
fig = make_subplots(
    rows=1,
    cols= 1,# 2,
    shared_xaxes=False,
    shared_yaxes=False,
    horizontal_spacing=0.02,
    column_titles=["Raw", "Preprocessed"],
)
fig.update_layout(
    layout,
    height=450,
)

# Only show a sample of spectra to avoid rendering issues
num_samples = len(chlamy_dataframe)# min(50, len(chlamy_dataframe))

plotted_wells = []
for i, dataframe_row in chlamy_dataframe.iterrows(): # .sample(num_samples).iterrows():
    # Get the raw and preprocessed spectra and its associated metadata
    raw_spectrum = chlamy_spectra[i]
    preprocessed_spectrum = preprocessed_chlamy_spectra[i]
    well_ID = dataframe_row["well_ID"]
    site = dataframe_row["site"]

    # Plot each raw spectrum
    fig.add_trace(
        go.Scatter(
            x=raw_spectrum.wavenumbers_cm1, # preprocessed_spectrum.wavenumbers_cm1
            y=raw_spectrum.intensities, # preprocessed_spectrum.intensities
            hovertext=f"{well_ID} | {site}",
            hoverinfo="text",
            legendgroup=well_ID,
            showlegend=False,
            marker={"color": color_map[well_ID]},
            opacity=0.5,
        ),
        row=1,
        col=1,
    )

    '''
    # Plot each preprocessed spectrum
    fig.add_trace(
        go.Scatter(
            x=preprocessed_spectrum.wavenumbers_cm1,
            y=preprocessed_spectrum.intensities,
            hoverinfo="skip",
            legendgroup=well_ID,
            showlegend=False,
            marker={"color": color_map[well_ID]},
            opacity=0.5,
        ),
        row=1,
        col=2,
    )'''
    plotted_wells.append(well_ID)

plotted_wells = pd.Series(plotted_wells)
# Plot fake traces for legend
for well_ID in plotted_wells.unique():
    fig.add_trace(
        go.Scatter(
            x=[None],
            y=[None],
            mode="lines",
            name=well_ID,
            line={"color": color_map[well_ID], "width":3},
            legendgroup=well_ID,
            showlegend=True,
        )
    )

# Configure plotly layout
fig.update_layout(layout, height=600)
fig.update_xaxes(title_text="Wavenumber (cm<sup>-1</sup>)", row=1, col=1)
fig.update_xaxes(title_text="Wavenumber (cm<sup>-1</sup>)", row=1, col=2)
fig.update_yaxes(title_text="Intensity", row=1, col=1)
fig.show()


In [6]:
import plotly.express as px
import numpy as np
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis



# Set number of LDA components
num_lda_components = 2

# Define groups and group labels
group_labels = {well:i for i, well in enumerate(chlamy_dataframe.well_ID.unique())}

# Filter DataFrame for Wasatch 785 nm group
instrument = "wasatch"
wavelength_nm = 785
grp = chlamy_dataframe.query("instrument == @instrument & λ_nm == @wavelength_nm")

# Compose training data
X = []
y = []
for _well_ID, label in group_labels.items():
    indices_in_group = grp.query("well_ID == @_well_ID").index.values
    spectra_in_group = [preprocessed_chlamy_spectra[i] for i in indices_in_group]
    for spectrum in spectra_in_group:
        X.append(spectrum.intensities)
        y.append(label)
X = np.array(X)
y = np.array(y)

# Construct, fit, and transform LDA
lda = LinearDiscriminantAnalysis(n_components=num_lda_components)
spectral_components = lda.fit(X, y).transform(X)
variances = lda.explained_variance_ratio_

# Add LDA components to DataFrame for plotting
source = pd.DataFrame(
    {
        "LD-1": spectral_components[:, 0],
        "LD-2": spectral_components[:, 1],
        "label": y,
    }
)
group_labels_inverted = {k: v for v, k in group_labels.items()}
source["Group"] = source["label"].map(group_labels_inverted)

# LDA plot
fig = px.scatter(
    source,
    x="LD-1",
    y="LD-2",
    color="Group",
    color_discrete_map=color_map,
    marginal_x="box",
    marginal_y="box",
)

# Configure plotly layout
fig.update_layout(
    layout,
    title=f"{instrument.capitalize()} | {wavelength_nm} nm",
    xaxis_title=f"LD-1 ({variances[0]:.1%})",
    yaxis_title=f"LD-2 ({variances[1]:.1%})",
    height=500,
    width=600,
    # xaxis={"scaleanchor": "y"},
)
fig.show()
